In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [2]:
# %% update data
import socket
from scripts.util.data.DataUpdater import update_server_main , update_laptop_main
from scripts.util.data.sqlConnector import update_sql_since , update_sql_dates

if socket.gethostname() == 'mengkjin-server':
    update_server_main()
else:
    update_laptop_main()
update_sql_since()
if False:
    update_sql_dates(20240101 , 20240229)

Use device name: NVIDIA GeForce RTX 4090
Merge Update Files
['/home/mengkjin/Workspace/SharedFolder/DB_updater.0.h5', '/home/mengkjin/Workspace/SharedFolder/DB_updater.1.h5']
Mon Mar 11 07:44:26 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./basic/calendar copying Done!
Mon Mar 11 07:44:35 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/concepts copying Done!
Mon Mar 11 07:44:35 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/description copying Done!
Mon Mar 11 07:44:54 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/industry copying Done!
Mon Mar 11 07:44:54 2024 : DB_updater.0.h5 > data/DB_data/DB_information/DB_information.h5/./stock/st copying Done!
Mon Mar 11 07:44:54 2024 : DB_updater.0.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag0 copying Done!
Mon Mar 11 07:44:54 2024 : DB_updater.0.h5 > data/DB_data/DB_labels/DB_labels.2024.h5/./10days/lag1 copy

/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


...

/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


/home/mengkjin/Workspace/learndl/scripts/util/data/sqlConnector.py:270: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df.fillna(np.nan)


Mon Mar 11 08:17:03 2024 : huatai/dl_factors since 20170101 to 20240311, total 28 periods(QE)
├── dongfang <HDF5 group "/dongfang" (8 members)>10          
│   ├── gp <HDF5 group "/dongfang/gp" (43 members)>
│   │   ├── 20240102 <HDF5 group "/dongfang/gp/20240102" (3 members)> (feature, secid, values)
│   │   ├── 20240103 <HDF5 group "/dongfang/gp/20240103" (3 members)> (feature, secid, values)
│   │   ├── ...... omitting 39 memebers
│   │   ├── 20240307 <HDF5 group "/dongfang/gp/20240307" (3 members)> (feature, secid, values)
│   │   └── 20240308 <HDF5 group "/dongfang/gp/20240308" (3 members)> (feature, secid, values)
│   ├── hfq_chars <HDF5 group "/dongfang/hfq_chars" (43 members)>
│   │   ├── 20240102 <HDF5 group "/dongfang/hfq_chars/20240102" (3 members)> (feature, secid, values)
│   │   ├── 20240103 <HDF5 group "/dongfang/hfq_chars/20240103" (3 members)> (feature, secid, values)
│   │   ├── ...... omitting 39 memebers
│   │   ├── 20240307 <HDF5 group "/dongfang/hfq_chars/20240307

In [2]:
# %% test a specific model
from run_model import main
main(process = 0 , rawname = 1 , resume = 0 , anchoring = 0)

24-03-14 07:45:48|MOD:run_model   |: Model Specifics:
24-03-14 07:45:48|MOD:run_model   |: Start Process [Load Data]!


--Process Queue : Data + Train + Test + Instance
--Start Training New!
--Model_name is set to resnet_lstm_30m!
{'verbosity': 2,
 'storage_type': 'mem',
 'precision': 'float',
 'batch_size': 10000,
 'model_name': 'resnet_lstm_30m',
 'model_module': 'resnet_lstm',
 'model_data_type': '30m',
 'model_num': 3,
 'beg_date': 20170103,
 'end_date': 99991231,
 'interval': 120,
 'input_step_day': 5,
 'test_step_day': 1,
 'MODEL_PARAM': {'hidden_dim': [64],
                 'seqlens': [{'day': 40, '30m': 30, '15m': 20, 'dms': 40}],
                 'tra_seqlens': [{'hist_loss': 40}],
                 'dropout': [0.1],
                 'enc_in': [True],
                 'enc_att': [False],
                 'rnn_type': ['lstm'],
                 'rnn_att': [False],
                 'rnn_layers': [2],
                 'dec_mlp_layers': [1],
                 'num_output': [1],
                 'kernel_size': [3, 3],
                 'hidden_as_factor': [False],
                 'ordered_param_group':

24-03-14 07:45:53|MOD:run_model   |: Finish Process [Load Data]! Cost 5.0Secs
24-03-14 07:45:53|MOD:run_model   |: Start Process [Train Model]!


Pre-Norming method of [30m] : [endpoint_division(False) , history_standardize(True)]


resnet_lstm_30m #0 @20170103 LoadData Cost   12.6Secs
FirstBite Ep#  0 : loss  1.00250, train-0.00243, valid 0.01302, max 0.0130, best 0.0130, lr1.0e-07
FirstBite Ep#  5 : loss  0.86859, train 0.14098, valid 0.13543, max 0.1354, best 0.1354, lr3.8e-03
FirstBite Ep# 10 : loss  0.82838, train 0.18834, valid 0.11802, max 0.1354, best 0.1354, lr1.3e-03
FirstBite Ep# 15 : loss  0.79074, train 0.23486, valid 0.10619, max 0.1354, best 0.1354, lr6.3e-04
FirstBite Ep# 20 : loss  0.77886, train 0.25002, valid 0.10261, max 0.1354, best 0.1354, lr1.3e-03


In [5]:
import os
del_path = './instance/tra_lstm2_day_ShortTest'
if os.path.exists(del_path): os.rmdir(del_path)
# rm -r ./instance/tra_lstm2_day_ShortTest #type: ignore

In [ ]:
# %% model data preprocessing
from data_preprocessing import main
main(confirm=1)